## Part 2 in the pandas df to SQL Server data pipeline

### NB: In Part 2, we assume that we've already run the Part 1 notebook (or equivalent scripts), the latter of which is intended to be run only *once*. However, the scripts, functions, and methods shown here can (and should) be re-used every time we have scraped more data and need to insert the new data into the SQL rental table. 

##### I.e.: the rental table has already been created within the craigslist database, and we have successfully inserted at least some data into the rental table!

##### If so, then we will do the following here in Part 2: check for the last date of inserted date--ie, MAX() of date_possted--and then filter the scraped data > the MAX()date in the SQL table, clean the data using the various data type transformations, deduplications, removing nulls, etc. 

#### As a final data filtering step, we need to double-check whether any of these rental listings have already been inserted into the rental table using a SQL query in which we can employ the IN() operator . For example: rental listings might be taken down and then re-posted, and these listings--albeit quite rare--could have the same listing ids as an older listing that we've already stored in the rental table, even though these "new" listings would initially seem to be unique listings based purely on their more recent date_posted datetime value!

#### After the data have been filtered and cleaned, we can then insert the new data into the table.

In [2]:
# imports-- file processing & json libraries
import os
import glob
import json

# data analysis libraries & SQL libraries
import numpy as np
import pandas as pd
# SQL ODBC for API connection between Python & SQL Server
import pyodbc

### Import all scraped data:

In [3]:
def recursively_import_all_CSV_and_concat_to_single_df(parent_direc, fn_regex=r'*.csv'):
    """Recursively search parent directory, and look up all CSV files.
    Then, import all CSV files to a single Pandas' df using pd.concat()."""
    path =  parent_direc # specify parent path of directories containing the scraped rental listings CSV data -- NB: use raw text--as in r'path...', or can we use the double-back slashes to escape back-slashes??
    df_concat = pd.concat((pd.read_csv(file, # import each CSV file from directory
                                        sep=',',encoding = 'utf-8'  
                                        ) for file in glob.iglob(
                                            os.path.join(path, '**', fn_regex), 
                                            recursive=True)), ignore_index=True)  # os.path.join helps ensure this concatenation is OS independent
    return df_concat

## Import Dataset
# import all scraped SF bay area rental listings data
scraped_data_path = r"D:\\Coding and Code projects\\Python\\craigslist_data_proj\\CraigslistWebScraper\\scraped_data\\sfbay"

df = recursively_import_all_CSV_and_concat_to_single_df(scraped_data_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29136 entries, 0 to 29135
Data columns (total 49 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   listing_urls             29136 non-null  object 
 1   ids                      28931 non-null  float64
 2   sqft                     22294 non-null  object 
 3   cities                   29019 non-null  object 
 4   prices                   28927 non-null  object 
 5   bedrooms                 28615 non-null  object 
 6   bathrooms                28927 non-null  object 
 7   attr_vars                28928 non-null  object 
 8   listing_descrip          28928 non-null  object 
 9   date_of_webcrawler       29021 non-null  object 
 10  kitchen                  29019 non-null  float64
 11  date_posted              28928 non-null  object 
 12  region                   29136 non-null  object 
 13  sub_region               29136 non-null  object 
 14  cats_OK               

### Determine last date (ie, MAX()) of the data stored in the rental table:

In [4]:
# Perform SQL query on the date_posted col to determine the most recent date of data stored in the table  
class SQL_Database:
    def __init__(self, path_for_SQL_config):

        with open(path_for_SQL_config,'r') as fh:
            config = json.load(fh)

        self.driver = config['driver']
        self.server = config['server']
        self.database = config['database']
        self.username = config['username']
        self.password = config['password']

        print(self.database)

    def determine_latest_date(self, sql_query):
        """Insert scraped Craigslist rental listings data (ie, the Pandas' dataframe)
        to SQL Server database 'rentals' table"""

        conn = pyodbc.connect(
        f'DRIVER={self.driver};'
        f'SERVER={self.server};'
        f'DATABASE={self.database};'
        f'UID={self.username};'
        f'PWD={self.password};'
        'Trusted_Connection=yes;'
        )

        # initialize cursor so we can execute SQL code
        cursor = conn.cursor() 

        # specify SQL query
        sql_query = sql_query 

        # perform query, and convert query results to Pandas' df
        max_date = pd.read_sql(sql_query, conn)

        conn.commit()

        cursor.close()
        conn.close()

        ## sanity check:
        print(f"Latest date of scraped data inserted into the SQL table:\n{max_date}")

        return max_date

# specify path to json file containing SQL configuration/username data
sql_config_path = "D:\\Coding and Code projects\\Python\\craigslist_data_proj\\CraigslistWebScraper\\SQL_config\\config.json" 

SQL_db = SQL_Database(sql_config_path)  # NB: be sure to pass in path to the json SQL configuration file so we can load in the needed username, password, and configuration data to be able to access the SQL database

# specify query to select the latest date based on date_posted:
query = "SELECT MAX(date_posted) AS latest_date FROM rental;"

latest_date = SQL_db.determine_latest_date(query)

craigslist
Latest date of scraped data inserted into the SQL table:
  latest_date
0        None


## Before we filter the scraped listings dataset (ie, dataframe), we need to transform the 'date_posted' column--ie, the one that we will filter--to a standardized datetime format

#### Why?: To ensure consistency and replicability for this data pipeline (ie, of the pandas' DataFrame to SQL Server table):

In [5]:
def transform_cols_to_datetime(df, col_to_convert):
    """Transform relevant column(s) to datetime using pd.to_datetime() method, and use infer_datetime_format=True to enable allow for datetime conversion using differing formats (ie, date_posted has a somewhat more precise format). """
    return pd.to_datetime(df[col_to_convert], infer_datetime_format=True)

# apply transformations to datetime for the 2 relevant cols:
df['date_of_webcrawler'] =  transform_cols_to_datetime(df,'date_of_webcrawler')
df['date_posted'] = transform_cols_to_datetime(df,'date_posted')

#sanity check
df[['date_posted', 'date_of_webcrawler']].head()

,date_posted,date_of_webcrawler
0,2021-12-27 17:45:00,2022-01-03
1,2022-01-03 00:49:00,2022-01-03
2,2022-01-03 00:20:00,2022-01-03
3,2021-12-10 13:16:00,2022-01-03
4,2021-12-19 02:14:00,2022-01-03


## Next, we need to filter the dataset to listings records whose date_posted dates are newer (read: greater than) the MAX() of the last date found from the query of the SQL rental table.

### To this end, we can convert the MAX() value from the SQL query to a str value, and then use this as the argument for the filter_df_since_specified_date() function, which filters on the 'date_posted' column: 

In [6]:
## next, convert this latest_date to a string value, so we can use this to filter the scraped dataframe dataset
def datetime_col_to_str_of_datetime(df, datetime_col):
    """Given datetime col from pandas' DataFrame,
    transform to a string of the datetime value."""
    return df[datetime_col].head(1).astype(str).reset_index().loc[0, datetime_col] 

# specify name of query result DataFrame and the corresponding datetime col:
latest_date, dt_col = latest_date, 'latest_date' 
#apply function using the 2 arguments shown above
latest_date_str = datetime_col_to_str_of_datetime(latest_date, dt_col)
# sanity check
print(f'The latest date among the scraped data stored in the SQL table is:\n{latest_date_str},\ndata type of this variable is (NB"" should be str): {type(latest_date_str)}')

The latest date among the scraped data stored in the SQL table is:
None,
data type of this variable is (NB"" should be str): <class 'str'>


### Finally, filter on the date_posted date, so that we only have the new listings data that we have not yet inserted into the SQL table:

In [7]:
if latest_date_str != "None":   # account for scenario in which *no data* has yet been inserted into the SQL table
    print(True)
else:
    print(False)

False


In [10]:
# filter the dataframe > MAX() of latest_date stored in SQL rental table
def filter_df_since_specified_date(df, target_date: str):
    """Filter the imported scraped dataset to all data newer than the specified date (as determined via the MAX(posted_date) query)."""
    if target_date != "None":   # account for scenario in which *no data* has yet been inserted into the SQL table
        df = df.loc[df['date_posted'] > target_date]  # filter to data greater than specified date
    else: 
        pass  # do not apply filter, sin`ce no data has yet been inserted into SQL database
    return df

# get all data since the latest stored data from SQL table (via the query on MAX(posted_date)) 
df = filter_df_since_specified_date(df, latest_date_str)

# sanity check
print(f"The newest scraped data not stored in the SQL table is--\n*NB: this should be an empty df if we have already stored all of the df's data into the SQL table*: \n\n{df['date_posted']}")

The newest scraped data not stored in the SQL table is--
*NB: this should be an empty df if we have already stored all of the df's data into the SQL table*: 

0       2021-12-27 17:45:00
1       2022-01-03 00:49:00
2       2022-01-03 00:20:00
3       2021-12-10 13:16:00
4       2021-12-19 02:14:00
                ...        
29131   2022-02-03 11:08:00
29132   2022-02-10 09:52:00
29133   2022-02-10 08:51:00
29134   2022-02-10 06:03:00
29135   2022-02-10 02:41:00
Name: date_posted, Length: 29136, dtype: datetime64[ns]


## Next, perform all additional data cleaning and wrangling features to prep the data for the SQL inserts, as in the Part 1 jupyter notebook

In [101]:
def remove_nulls_list(df, list_of_cols):
    """Remove rows that do not have price, city name, kitchen, sqft, or listing ID data, as these are essential variables in this rental listings dataset."""
    return df.dropna(subset=list_of_cols)

list_cols_to_remove_nulls = ['prices', 'ids', 'sqft', 'kitchen', 'cities']  
df = remove_nulls_list(df, list_cols_to_remove_nulls)

# sanity check
print(f"Remaining price, listing id, sqft, kitchen, & city name nulls: \n{df[list_cols_to_remove_nulls].isnull().sum()}")

Remaining price, listing id, sqft, kitchen, & city name nulls: 
prices     0
ids        0
sqft       0
kitchen    0
cities     0
dtype: int64


In [102]:
def clean_split_city_names(df, address_critera: list, neighborhood_criteria:list, split_city_delimiters: list, incorrect_city_names:dict, cities_not_in_region:dict, cities_that_need_extra_cleaning:dict):
    """Clean city names data in several ways:
    a.) Remove extraneous address & neighborhood data placed in the city names HTML object, such as 'Rd', 'Blvd', or 'Downtown'.
    b.) Unsplit city names data that are split via ',' & '/' delimiters.
    c.) Replace abbreviated or mispelled city names, and remove city names that do not exist within the SF Bay Area (e.g., 'Redding').
    d.) Remove any digits/integers within the city names data--ie, by using a '\d+' regex as the argument of str.replace() and replace it with empty strings.
    e.) Remove any city names records thast are left with merely empty strings (ie, the other steps removed all data for that given cities record).
    f.) Remove any whitespace to avoid the same city names from being treated as different entities by Pandas, Python, or SQL. 
    g.) Use str.capwords() to capitalize words (ie, excluding apostrophes).
    h.) Replace city names that are mispelled after having removed various street and neighborhood substrings such as 'St' or 'Ca'--e.g., '. Helena' should be 'St. Helena'. """
    # specify extraneous street & address data (e.g., 'Rd') that we want to remove from the city names column:
    addr_criteria = '|'.join(address_critera) # Join pipe ('|') symbols to address list so we can str.split() on any one of these criteria (ie, 'or' condition splitting on each element separated by pipes):
    # specify extraneous neighborhood criteria we should also remove from col
    nbhood_criteria = '|'.join(neighborhood_criteria) # remove neighborhood names as well as state abbreviation (shown on website as 'Ca') that is shown without the usual comma delimiter!
    # b.) specify delimiters we need to refer to un-split city names:
    split_city_delimiters = '|'.join(split_city_delimiters) # join pipes to delimiters so we can use str.split() based on multiple 'or' criteria simultaneously
    # clean city names data by removing extraneous address & neighborhood data, and unsplitting city names based on ',' & '\' delimiters
    df['cities'] =  df['cities'].str.split(addr_criteria).str[-1].str.replace(nbhood_criteria, '', case=True).str.lstrip()
    df['cities'] = df['cities'].str.split(split_city_delimiters).str[0] #unsplit city names based on comma or forward-slash delimiters
    # c.) replace specific abbreviated or mispelled city names, and remove cities that are not actually located in the sfbay region:
    df = df.replace({'cities':incorrect_city_names}) # replace mispelled & abbreviated city names
    df = df.replace({'cities':cities_not_in_region})  # remove (via empty string) cities that are not actually located in the sfbay region
    # d.) Remove digits/integer-like data from cities column:
    df['cities'] = df['cities'].str.replace('\d+', '')  # remove any digits by using '/d+' regex to look up digits, and then replace with empty string
    # e.) Remove any rows that have empty strings or null values for cities col (having performed the various data filtering and cleaning above)
    df = df[df['cities'].str.strip().astype(bool)] # remove rows with empty strings (ie, '') for cities col 
    df = df.dropna(subset=['cities']) # remove any remaining 'cities' null records
    # f.) Remove whitespace
    df['cities'] = df['cities'].str.strip() 
    # g.) capitalize the city names using str.capwords() 
    df['cities'] = df['cities'].str.split().apply(lambda x: [val.capitalize() for val in x]).str.join(' ')
    # h) Replace city names that are mispelled after having removed various street and neighborhood substrings such as 'St' or 'Ca'--e.g., '. Helena' should be 'St. Helena' & 'San los' should be 'San Carlos'. Also, remove any non-Bay Area cities such as Redding:
    # df['cities'] = df['cities'].str.lower() # transform all records to lower-case, for ease of cleaning the data
    df = df.replace({'cities':cities_that_need_extra_cleaning})
    return df

# specify various address and street name that we need to remove from the city names 
address_criteria = ['Boulevard', 'Blvd', 'Road', 'Rd', 'Avenue', 'Ave', 'Street', 'St', 'Drive', 'Dr', 'Real', 'E Hillsdale Blvd'] 
# specify various extraneous neighborhood names such as 'Downtown' 
neighborhood_criteria = ['Downtown', 'Central/Downtown', 'North', 'California', 'Ca.', 'Bay Area', 'St. Helena', 'St', 'nyon', 
'Jack London Square', 'Walking Distance To', 'El Camino', 'Mendocino County', 'San Mateo County', 'Alameda County', 'Rio Nido Nr', 'Mission Elementary', 
'Napa County', 'Golden Gate', 'Jennings', 'South Lake Tahoe', 'Tahoe Paradise', 'Kingswood Estates', 'South Bay', 'Skyline', 'San Antonio Tx', 
'East Bay', 'Morton Dr'] 

# specify what delimiters we want to search for to unsplit the split city names data:
split_city_delimiters =  [',', '/', ' - ']
# specify dictionary of abbreviated & mispelled cities:
incorrect_city_names = {'Rohnert Pk':'Rohnert Park', 'Hillsborough Ca': 'Hillsborough', 'South Sf': 'South San Francisco', 'Ca':'', 'East San Jose':'San Jose', 'Vallejo Ca':'Vallejo', 'Westgate On Saratoga .':'San Jose', 'Bodega':'Bodega Bay', 'Briarwood At Central Park':'Fremont', 'Campbell Ca':'Campbell', 'Almaden':'San Jose', '.':'', 'East Foothills':'San Jose', 'Lake County':'', 'West End':'Alameda', 'Redwood Shores':'Redwood City'}

# specify dictionary of cities that are not located in sfbay (ie, not located in the region):
cities_not_in_region = {'Ketchum':'', 'Baypoinr':'', 'Quito': '', 'Redding':'', 'Bend' :''}

# specify dictionary of city names that are mispelled after having removed various street and neighborhood substrings:
cities_that_need_extra_cleaning = {'. Helena': 'St. Helena', '. Helena Deer Park': 'St. Helena', 'San Los':'San Carlos', 'Tro Valley':'Castro Valley', 'Rohnert Pk':'Rohnert Park',
'Pbell':'Campbell', 'Pbell Ca':'Campbell', 'American Yon':'American Canyon', 'Millbrae On The Burlingame Border':'Millbrae', 'Ockton Ca': 'Stockton', '. Rohnert Park': 'Rohnert Park'}

# clean city names data:
df = clean_split_city_names(df, address_criteria, neighborhood_criteria, split_city_delimiters, incorrect_city_names, cities_not_in_region, cities_that_need_extra_cleaning)
# sanity check
print(f"Sanity check--after cleaning the city names, let's examine some of the cleaned data: {df.cities.value_counts().tail(10)}")

Sanity check--after cleaning the city names, let's examine some of the cleaned data: Bay Point            1
Bodega Bay           1
Pleasant Hill        1
Oakley               1
Tracy                1
Near Mount Lassen    1
Suisun City          1
Felton               1
Greenbrae            1
Middletown           1
Name: cities, dtype: int64


C:\Users\Kevin Allen\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
C:\Users\Kevin Allen\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: The default value of regex will change from True to False in a future version.


In [103]:
def transform_cols_to_indicators(df, list_of_cols):
    """ Transform relevant attribute columns to numeric, and specify NaNs for any missing or non-numeric data."""
    df[list_of_cols] = df[list_of_cols].astype('uint8', errors='ignore') # convert any missing data to NaN 
    df[list_of_cols] = df[list_of_cols].fillna(0) # impute any NaN values with 0s since we can be quite certain that any rental listings not explicitly specifying a specific property type, amenity, or attribute do not actually contain said attribute, amenity, or are not said propery type  
    print(f"Sanity check: The data types of {list_of_cols} are now: \n{df[list_of_cols].dtypes}") # sanity check on columns' data types
    return df

# specify a list of cols to convert to numeric -- # since there are many cols we want to transform to indicator variables, it's easier to simply drop the few cols that comprise str (aka, object) data 
cols_to_indicators = df.drop(columns =['ids', 'listing_urls', 'region', 'sub_region', 'cities', 'attr_vars', 'listing_descrip', 'sqft', 'prices', 'bedrooms', 'bathrooms', 'date_posted', 'date_of_webcrawler']) 
cols_to_indicators_lis = list(cols_to_indicators.columns)
cols_to_indicators = [] # free space

df = transform_cols_to_indicators(df, cols_to_indicators_lis)

cols_to_indicators_lis = [] # free space

Sanity check: The data types of ['kitchen', 'cats_OK', 'dogs_OK', 'wheelchair_accessible', 'laundry_in_bldg', 'no_laundry', 'washer_and_dryer', 'washer_and_dryer_hookup', 'laundry_on_site', 'full_kitchen', 'dishwasher', 'refrigerator', 'oven', 'flooring_carpet', 'flooring_wood', 'flooring_tile', 'flooring_hardwood', 'flooring_other', 'apt', 'in_law_apt', 'condo', 'townhouse', 'cottage_or_cabin', 'single_fam', 'duplex', 'flat', 'land', 'is_furnished', 'attached_garage', 'detached_garage', 'carport', 'off_street_parking', 'no_parking', 'EV_charging', 'air_condition', 'no_smoking'] are now: 
kitchen                    uint8
cats_OK                    uint8
dogs_OK                    uint8
wheelchair_accessible      uint8
laundry_in_bldg            uint8
no_laundry                 uint8
washer_and_dryer           uint8
washer_and_dryer_hookup    uint8
laundry_on_site            uint8
full_kitchen               uint8
dishwasher                 uint8
refrigerator               uint8
oven    

In [104]:
# also, transform kitchen var separately, since this tends to otherwise convert to float:
df = transform_cols_to_indicators(df, 'kitchen')

Sanity check: The data types of kitchen are now: 
uint8


In [105]:
# re: # of bathrooms data, transform any records containing 'shared' or 'split' to 1
# Why?: Because we can assume that any rental units comprising a 'shared' bathroom is essentially 1 bathroom
def transform_shared_and_split_to_ones(df, col_to_transform):
    """Transform any records (from given col) containing the string values of 'shared' or 'split' to a value of 1."""
    # transform col to object, so we can use Python str methods to transform the data
    df[col_to_transform] = df[col_to_transform].astype('object') 
    bedroom_replace_criteria = ['shared', 'split']
    bedroom_replace_criteria = '|'.join(bedroom_replace_criteria) # join pipe symbols so we can use str.replace() on multiple 'or' conditions simultaneously 
    return df[col_to_transform].str.replace(bedroom_replace_criteria,'1')

# clean bathrooms data by replacing the 'split' and 'shared' string values:
df['bathrooms'] = transform_shared_and_split_to_ones(df, 'bathrooms')

#sanity check
print(f"Sanity check: \n{df['bathrooms'].value_counts()}")

Sanity check: 
1      2688
2      1107
1.5     134
2.5      80
3        58
3.5       8
4         6
6         2
9+        1
Name: bathrooms, dtype: int64


C:\Users\Kevin Allen\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == '__main__':


In [106]:
# replace any ambiguous # of bathrooms data--such as '9+' with empty strings (ie, essentially nulls) 
def replace_ambiguous_data_with_empty_str(df, col_to_transform):
    """Replace ambiguous rows of data (ie, any containing a plus sign) for bathrooms col with empty strings"""
    return df[col_to_transform].str.replace(r'\+', '')  # use str.replace() to use a regex to search for plus signs, and in effect remove these by replacing them with empty strings 

df['bathrooms']  = replace_ambiguous_data_with_empty_str(df, 'bathrooms')
# sanity check
print(f"New value counts for bathrooms data--having cleaned ambiguous records: \n{df['bathrooms'].value_counts()}")

New value counts for bathrooms data--having cleaned ambiguous records: 
1      2688
2      1107
1.5     134
2.5      80
3        58
3.5       8
4         6
6         2
9         1
Name: bathrooms, dtype: int64


C:\Users\Kevin Allen\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [107]:
# next, remove any bathroom or bedroom nulls:
def remove_bedroom_and_br_nulls(df):
    return df.dropna(subset=['bedrooms', 'bathrooms'])

df = remove_bedroom_and_br_nulls(df)

# sanity check
print(f"Remaining bedroom & bathroom nulls: \n{df[['bedrooms', 'bathrooms']].isnull().sum()}")

Remaining bedroom & bathroom nulls: 
bedrooms     0
bathrooms    0
dtype: int64


## Final Data Cleaning step-- cont'd:

## If there are any rental listing duplicates between the rental table and the datetime-filtered DataFrame, then we clearly need to provide an additional filter on this DataFrame:

## Namely: filter out any of these duplicate listings from the datetime-filtered pandas' DataFrame based on the above query results:

In [108]:
# transform bathrooms data to float
# Why float?: Because some listings specify half bathrooms--e.g., 1.5 denotes one-and-half bathrooms. Re: ids, integer data type not store the entire id value due to maximum (byte) storage constraints. 
def transform_cols_to_float(df, col_to_transform):
    return df[col_to_transform].astype('float32')

# convert bathrooms to float:
df['bathrooms'] = transform_cols_to_float(df, 'bathrooms')    

#sanity check
print(f"Sanity check on data type of bathrooms data: {df['bathrooms'].dtype}")

Sanity check on data type of bathrooms data: float32


In [110]:
def transform_cols_to_int(df, list_of_cols_to_num):
    """ Transform relevant attribute columns to numeric.
    NB: Since the scraped 'prices' data can contain commas, we need to use str.replace(',','') to remove them before converting to numeric."""
    df['prices'] = df['prices'].str.replace(",","") # remove commas from prices data (e.g.: '2500' vs '2,500')
    # clean sqft data --remove all non-numeric data
    df['sqft'] = df['sqft'].astype(str).str.replace(r'\D+', '', regex=True) # remove all non-numeric data from 'sqft' col by using regex to replace any non-numeric data from col to null ('NaN') values via the str.replace() Pandas method
    df['sqft'] = df['sqft'].replace(r'^\s*$', np.nan, regex=True)  # replace all empty str sqft values with  null ('NaN') values 
    # clean prices data-- remove any records posted with sqft instead of price data
    df = df[~df.prices.str.contains("ft2")] # remove listings records with incorrectly posted prices data 
    # remove rows with any remaining null rows wrt list of cols (ie, sqft, prices, etc.) (so we can readily convert to int):
    df = df.dropna(subset=list_of_cols_to_num) # remove rows with null data 
    # finally, convert all cols from list to 'int64' integer data type:
    df[list_of_cols_to_num] = df[list_of_cols_to_num].astype('int64') # use int64 due to a) the long id values & b.) the occasional null values contained within the sqft col
    print(f"Sanity check: The data types of {list_of_cols_to_num} are now: \n{df[list_of_cols_to_num].dtypes}") # sanity check on columns' data types
    return df


# specify a list of cols to convert to integer
cols_to_int = df[['bedrooms', 'prices', 'ids', 'sqft']]
cols_to_int_lis = list(cols_to_int.columns)  # convert relevant cols to list of col names

cols_to_int = [] # free space

df = transform_cols_to_int(df, cols_to_int_lis)

Sanity check: The data types of ['bedrooms', 'prices', 'ids', 'sqft'] are now: 
bedrooms    int64
prices      int64
ids         int64
sqft        int64
dtype: object


In [111]:
def deduplicate_df(df):
    """Remove duplicate rows based on listing ids"""
    return df.drop_duplicates(keep='first', subset = ['ids'])

df = deduplicate_df(df)

# sanity check -- 
clist_duplicate_ids_check = df[df.duplicated("ids", keep= False)]
print(f"There should be no remaining duplicate listing ids (ie, 0 rows): \n{clist_duplicate_ids_check.shape[0]}")  # check that number of duplicate rows is false (ie, wrt duplicate listing ids)

# free memory
clist_duplicate_ids_check = [] 

There should be no remaining duplicate listing ids (ie, 0 rows): 
0


In [112]:
def remove_col_with_given_starting_name(df, col_starting_name):
    """Remove each column from df that has a given starting name substring."""
    return df.loc[:, ~df.columns.str.startswith(col_starting_name)] 

# remove 'Unnamed' columns, which might be imported errouneously via pd.read_csv()
df = remove_col_with_given_starting_name(df, 'Unnamed')

# remove listing_urls column since we do not want to store these data into the SQL Server table-- why?: a.) because listing urls are not relevent to rental prices and b.) the listing urls quickly become invalid or dead links, so we have no need to refer back to them at this stage in the webscraping project.
df = remove_col_with_given_starting_name(df, 'listing_urls')

# remove listing_descrip col since we do not want to store these data in SQL table either;
df = remove_col_with_given_starting_name(df, 'listing_descrip')

# sanity check
print(f"Sanity check--The remaining columns in the dataset are:\n {df.columns}")

Sanity check--The remaining columns in the dataset are:
 Index(['ids', 'sqft', 'cities', 'prices', 'bedrooms', 'bathrooms', 'attr_vars',
       'date_of_webcrawler', 'kitchen', 'date_posted', 'region', 'sub_region',
       'cats_OK', 'dogs_OK', 'wheelchair_accessible', 'laundry_in_bldg',
       'no_laundry', 'washer_and_dryer', 'washer_and_dryer_hookup',
       'laundry_on_site', 'full_kitchen', 'dishwasher', 'refrigerator', 'oven',
       'flooring_carpet', 'flooring_wood', 'flooring_tile',
       'flooring_hardwood', 'flooring_other', 'apt', 'in_law_apt', 'condo',
       'townhouse', 'cottage_or_cabin', 'single_fam', 'duplex', 'flat', 'land',
       'is_furnished', 'attached_garage', 'detached_garage', 'carport',
       'off_street_parking', 'no_parking', 'EV_charging', 'air_condition',
       'no_smoking'],
      dtype='object')


## Final Data cleaning Step:

## Determine if there are any rental listing duplicates between the rental table and the datetime-filtered DataFrame.

## If any such duplicates exist, then we need to filter these duplicates out of the datetime-filtered DataFrame.

## Implement SQL Query to determine whether there any duplicates in the datetime-filtered DataFrame relative to the SQL rental table:

### I.e., use a query with an IN() operator to check for a list of all listing id values from the datetime-filtered DataFrame, and compare those values with all stored rental listings data in the rental table: 

In [82]:
df = df.astype(str)  # convert dataframe to string, so we can enable pandas' df data to be more compatible with pyodbc library 
df_filtered_ids = df['ids'].to_list()  # obtain a list of all ids from the filtered df

# get various '?' SQL placeholders (ie, to prevent SQL injections) and a comma for each elements of the list of ids values--NB: reference the len() of the df_filtered_ids list to get the proper number of placeholders
q_placeholders = ','.join("?" * len(df_filtered_ids))

df_filtered_ids

['7444562463',
 '7444561871',
 '7444562359',
 '7444562620',
 '7444561488',
 '7444562067',
 '7444555242',
 '7444544140',
 '7444543866',
 '7444543404',
 '7442184532',
 '7442468806',
 '7444521438',
 '7444509101',
 '7444501200',
 '7444496508',
 '7443602840',
 '7441918360',
 '7444496033',
 '7444495690',
 '7444493139',
 '7444492966',
 '7444492538',
 '7444492451',
 '7444492377',
 '7444492290',
 '7444492200',
 '7444490137',
 '7444489592',
 '7444489451',
 '7444486790',
 '7444485390',
 '7444485181',
 '7444484883',
 '7444483834',
 '7444482612',
 '7444481977',
 '7444481322',
 '7444481312',
 '7444480953',
 '7444479681',
 '7444477596',
 '7444476831',
 '7444476749',
 '7444476668',
 '7444476228',
 '7442290175',
 '7444474276',
 '7444467806',
 '7444465445',
 '7444465386',
 '7444461161',
 '7444458960',
 '7444459780',
 '7444459388',
 '7444459379',
 '7444459265',
 '7444456739',
 '7444459095',
 '7444457620',
 '7444457079',
 '7444456924',
 '7444456742',
 '7444456191',
 '7444455129',
 '7444451121',
 '74444506

In [115]:
# verify whether any of the listing id's in the filtered DataFrame dataset are already stored within the rental listing dataset

class SQL_Database:
    def __init__(self, path_for_SQL_config):

        with open(path_for_SQL_config,'r') as fh:
            config = json.load(fh)

        self.driver = config['driver']
        self.server = config['server']
        self.database = config['database']
        self.username = config['username']
        self.password = config['password']

        print(self.database)

    def check_for_listing_ids_via_SQL_in_operator(self, df, target_date):
        """Insert scraped Craigslist rental listings data (ie, the Pandas' dataframe)
        to SQL Server database 'rental' table"""

        # establish connection to SQL Server database-specify login credentials:
        try:  # try to establish connection to SQL Server table via pyodbc connector
            conn = pyodbc.connect(
            f'DRIVER={self.driver};'
            f'SERVER={self.server};'
            f'DATABASE={self.database};'
            f'UID={self.username};'
            f'PWD={self.password};'
            'Trusted_Connection=yes;'
            )
        
        except pyodbc.Error as err:  # account for possible pyodbc SQL Server connection error
            print("Python was not able to connect to SQL server database and. Please try again.") 


        # initialize cursor so we can execute SQL code
        cursor = conn.cursor() 

        ## Perform SQL query on rental table to determine whether there are any rental listing duplicates--ie, listing ids from the filtered DataFrame that have already been inserted into the rental table--via SQL IN operator:
        
        # NB: *only* perform SQL query if at least some data has been inserted into SQL table--we can check whether any data exists based on whether the target_date has a value other than "None":
        if target_date != "None":   # account for scenario in which *no data* has yet been inserted into the SQL table, by ensuring the target_date (ie, latest_date_str) is not equal to "None"

            ## get a list of all listing ids from the datetime-filtered dataframe:
            # df = df.astype(str)  # convert dataframe to string, so we can enable pandas' df data to be more compatible with pyodbc library 
            df_filtered_ids = df['ids'].to_list()  # obtain a list of all ids from the filtered df

            # get various '?' SQL placeholders (ie, to prevent SQL injections) and a comma for each elements of the list of ids values--NB: reference the len() of the df_filtered_ids list to get the proper number of placeholders
            
            # q_placeholders = ", ".join(["?"] * len(df_filtered_ids))   # get '?' placeholders and comma for each id element from the df_filtered_ids list

            # q_placeholders = ",".join("?" * len(df_filtered_ids))  # get '?' placeholders and comma for each id element from the df_filtered_ids list


            ## specify query, and add placeholders in between each value being checked via the IN operator
            # sql_query = """SELECT * FROM rental WHERE listing_id IN ("""+ q_placeholders + ") "

            # sql_query = "SELECT * FROM rental WHERE listing_id IN (%s)" % df_filtered_ids

            q_placeholders = ','.join(['?']*len(df_filtered_ids))

            sql_query = """SELECT * FROM rental WHERE listing_id IN ({q_placeholders});"""

            id_query_duplicates = cursor.execute(sql_query, df_filtered_ids).fetchall()



            # sql_query = 'SELECT * FROM rental WHERE listing_id IN ({})'.format(', '.join(['?' for _ in df_filtered_ids]))


            # sql_query = "SELECT * FROM rental WHERE listing_id IN (%s)" % q_placeholders


            # sql_executable = cursor.execute(sql_query, df_filtered_ids)

            # id_query_duplicates = cursor.fetchall()

            # sql_query = cursor.execute("""SELECT * FROM rental WHERE listing_id IN {}""".format(q_placeholders), df_filtered_ids)


            ## use Pandas' read_sql() method to parse the query, and use the id values of the df--ie, from the df_filtered_ids list--as the argument of the IN operator: 
            
            # id_query_duplicates = pd.read_sql(sql_query, con=conn, params=[df_filtered_ids])  # query for any listings that have duplicate id's from the rental table relative to the 
            
            # sanity check-- Are there any duplicate listings in the SQL table vs the datetime-filtered DataFrame?
            print(f'Duplicate listings from the datetime-filtered dataframe relative to the SQL rental table:\n{id_query_duplicates}')        
            
            cursor.close()
            conn.close()

            return id_query_duplicates
        
        else:  # ie: if *no data* has yet been inserted into SQL table, then do nothing w/ this method
            pass

# specify path to json file containing SQL configuration/username data
sql_config_path = "D:\\Coding and Code projects\\Python\\craigslist_data_proj\\CraigslistWebScraper\\SQL_config\\config.json" 

SQL_db = SQL_Database(sql_config_path)  # NB: be sure to pass in path to the json SQL configuration file so we can load in the needed username, password, and configuration data to be able to access the SQL database
# perform query to check for any listings that are duplicate between the data in the rental table and the datetime-filtered dataframe 
id_query_duplicates = SQL_db.check_for_listing_ids_via_SQL_in_operator(df, latest_date_str)   # only apply query if latest_date_str is *not* "None", to ensure at least some data has been inserted into SQL table before trying to perform the query on it!

craigslist


ProgrammingError: ('42000', '[42000] [Microsoft][SQL Server Native Client 11.0]Syntax error, permission violation, or other nonspecific error (0) (SQLPrepare)')

In [48]:
# filter dataframe to remove any duplicate ids
def remove_duplicate_ids_relative_to_SQL_table(df, id_query_duplicates):
    """Remove any records from df whose ids are already inserted into (ie, duplicates of) the rental SQL table, as given by the id_query_duplicates query results"""
    if id_query_duplicates is not None:
        id_query_duplicates_list = id_query_duplicates['listing_id'].astype(float).apply(int).to_list() # derive a list of duplicate ids from the query results' listing_id column
        filtered_df = df[~df['ids'].isin(id_query_duplicates_list)]  # remove (ie, filter out) all duplicate ids using the negation of isin() 
        return filtered_df
    else:   # do *not* apply filter if no data has yet been inserted into SQL table (ie, if id_query_duplicates is None)
        return df
    

# apply the filter to remove duplicate ids, based on the query results (ie, id_query_duplicates)
df = remove_duplicate_ids_relative_to_SQL_table(df, id_query_duplicates)

df.info() # sanity check

NameError: name 'id_query_duplicates' is not defined

## Final Step-- Data pipeline and data ingestion: 

## Insert the cleaned/filtered/deduplicated pandas' DataFrame into SQL rental table:

#### NB: When using pyodbc to insert data from a dataframe into SQL Server table, we *may* need to transform all of the data from the dataframe to string, if SQL Server returns an error during an insertion attempt. This is because pyodbc will typically transform string values of numeric to SQL numeric data types such as int or float. 

#### Regardless, when using pyodbc to insert data into a SQL Server table, *always* use the  execute_many = True option so that the INSERT INTO statements will not be called upon as frequently. Ie, execute_many helps ensure that there will not be separate inserts for each row, as this is entirely unneccessary and extremely inefficient.

### Next, insert all scraped data into the rental SQL Server table:



### Insert scraped data from Pandas' dataframe to SQL Server rental table via pyodbc:

In [114]:
class SQL_Database:
    def __init__(self, path_for_SQL_config):

        with open(path_for_SQL_config,'r') as fh:
            config = json.load(fh)

        self.driver = config['driver']
        self.server = config['server']
        self.database = config['database']
        self.username = config['username']
        self.password = config['password']

        print(self.database)

    def insert_df_to_SQL_ETL(self, df):
        """Insert scraped Craigslist rental listings data (ie, the Pandas' dataframe)
        to SQL Server database 'rental' table"""

        # establish connection to SQL Server database-specify login credentials:
        try:  # try to establish connection to SQL Server table via pyodbc connector
            conn = pyodbc.connect(
            f'DRIVER={self.driver};'
            f'SERVER={self.server};'
            f'DATABASE={self.database};'
            f'UID={self.username};'
            f'PWD={self.password};'
            'Trusted_Connection=yes;'
            )
        
        except pyodbc.Error as err:  # account for possible pyodbc SQL Server connection error
            print("Python was not able to connect to SQL server database and. Please try again.") 


        # initialize cursor so we can execute SQL code
        cursor = conn.cursor() 

        cursor.fast_executemany = True  # speed up data ingesting by reducing the numbers of calls to server for inserts

        # convert all variables from dataframe to str to avoid following SQL Server pyodbc error: 'ProgrammingError: ('Invalid parameter type.  param-index=2 param-type=function', 'HY105')'
        # df = df.astype(str) # convert all df variables to str for ease of loading data into SQl Server table
        
        # insert scraped data from df to SQL table-- iterate over each row of each df col via .itertuples() method
        
        # Get the number of needed '?' placeholders by looking up the # of cols (ie, len()) of the dataframe), and use .join() to have each question mark separated by commas (NB: these placeholders are used to mitigate--and ideally avoid altogether--SQL injections)
        q_mark_str = ','.join('?'*len(df.columns))  
         

        # specify INSERT INTO SQL statement--iterate over each row in df, and insert into SQL database:
        for row in df.itertuples():  # iterate over each row from df
            cursor.execute(f"""INSERT INTO rental (listing_id, sqft, city, price, bedrooms, bathrooms, attr_vars,
            date_of_webcrawler, kitchen, date_posted, region, sub_region, cats_OK, dogs_OK, wheelchair_accessible,laundry_in_bldg, no_laundry, 
            washer_and_dryer, washer_and_dryer_hookup, laundry_on_site, full_kitchen, dishwasher, refrigerator,
            oven,flooring_carpet, flooring_wood, flooring_tile, flooring_hardwood, flooring_other,apt, in_law_apt, condo, townhouse, cottage_or_cabin, single_fam, duplex, flat, land, is_furnished, attached_garage,
            detached_garage, carport, off_street_parking, no_parking, EV_charging, air_condition, no_smoking) 
            VALUES ({q_mark_str})""",
            (row.ids,
            row.sqft,
            row.cities,
            row.prices, 
            row.bedrooms,
            row.bathrooms,
            row.attr_vars, 
            row.date_of_webcrawler,
            row.kitchen,
            row.date_posted,
            row.region,
            row.sub_region,
            row.cats_OK,
            row.dogs_OK,
            row.wheelchair_accessible,
            row.laundry_in_bldg, 
            row.no_laundry,
            row.washer_and_dryer,
            row.washer_and_dryer_hookup,
            row.laundry_on_site,
            row.full_kitchen,
            row.dishwasher,
            row.refrigerator,
            row.oven,
            row.flooring_carpet,
            row.flooring_wood,
            row.flooring_tile,
            row.flooring_hardwood,
            row.flooring_other,
            row.apt, 
            row.in_law_apt,
            row.condo,
            row.townhouse,
            row.cottage_or_cabin,
            row.single_fam, 
            row.duplex,
            row.flat,
            row.land,
            row.is_furnished,
            row.attached_garage,
            row.detached_garage,
            row.carport,
            row.off_street_parking,
            row.no_parking,
            row.EV_charging,
            row.air_condition,
            row.no_smoking)
            )
            
        # save and commit changes to database
        conn.commit()

        # # sanity check-- ensure some data has been inserted into new SQL table
        sql_table_count_records = conn.execute("""SELECT COUNT(*) FROM rental;""").fetchall()
        print(f"The number of records stored in the SQL table is: {sql_table_count_records[0]}")     
        
        sql_query_for_record_samples = conn.execute("""SELECT TOP 3 * FROM rental;""").fetchall() # check out several of the records
        print(f"\nA few of the inserted records are: {sql_query_for_record_samples}")

        cursor.close()
        conn.close()

# specify path to json file containing SQL configuration/username data
sql_config_path = "D:\\Coding and Code projects\\Python\\craigslist_data_proj\\CraigslistWebScraper\\SQL_config\\config.json" 

SQL_db = SQL_Database(sql_config_path)  # NB: be sure to pass in path to the json SQL configuration file so we can load in the needed username, password, and configuration data to be able to access the SQL database
# Ingest data from pandas' dataframe to SQL server--data pipeline: 
SQL_db.insert_df_to_SQL_ETL(df)


craigslist
The number of records stored in the SQL table is: (15877, )

A few of the inserted records are: [(7376737280, 10500, 'Daly City', 2295, 1, Decimal('1.0'), 'apartment\nlaundry in bldg\noff-street parking\nrent period: monthly', datetime.datetime(2021, 10, 12, 0, 0), 1, datetime.datetime(2021, 9, 6, 21, 54), 'sfbay', 'pen', 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0), (7377009152, 23360, 'Burlingame', 1650, 0, Decimal('1.0'), 'cats are OK - purrr\nflooring: wood\napartment\nlaundry in bldg\nno smoking\noff-street parking\nrent period: monthly', datetime.datetime(2021, 10, 12, 0, 0), 1, datetime.datetime(2021, 9, 7, 12, 5), 'sfbay', 'pen', 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1), (7377930752, 6500, 'Redwood City', 3550, 1, Decimal('1.0'), 'EV charging\nair conditioning\napplication fee details: $38 credit/background check\ncats are OK - purrr\ndogs are O